#Importación de librerías

In [0]:
import numpy as np
import pandas as pd
from scipy import stats
import networkx as nx

filter = "square"
filter_square = [64,90,74,78,73,107,79]
filter_district = filter_square + [78, 84, 77, 79, 76, 82, 73, 72, 63, 62, 61]
if filter=="square":
  retiro_stations = filter_square
  file = '201808_Usage_Bicimad.json.square'
else:
  retiro_stations = filter_district
  file = '201808_Usage_Bicimad.json.district'
  


#Funciones

## Funciones auxiliares

In [0]:
#Inventario de funciones

'''
  Función para calcular geo distancias.
  Recibe la longitud y latitud de dos puntos
  Devuelve la distancia.
'''
def calculateDistance(lat1, lon1, lat2, lon2):
  from math import sin, cos, sqrt, atan2, radians
  
  R = 6373.0 # approximate radius of earth in km

  lat1 = radians(lat1)
  lon1 = radians(lon1)
  lat2 = radians(lat2)
  lon2 = radians(lon2)

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))
  return R * c


'''
  Función para calcular velocidades basado en la información de los Tracks del viaje.
  Recibe el DataFrame de Viajes con los Tracks
  Quitaremos los tramos en los que speed = 0
  Devuelve un DataFrame con:
    oid => Identificador del viaje
    travel_time_track => Suma de Tiempos
    avg_speed_track => Calculado a través de la distancia total / tiempo total
    avg_speed => Media de los valores speed
'''
def calculateSpeed(df):
  import datetime
  print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'), 'Ini')
  oids = df['oid'].unique()
  df = pd.DataFrame(columns=['oid', 'travel_time_track', 'avg_speed_track', 'avg_speed'])
  for oid in oids:
    s = 0.0
    t = 0.0
    seconds_ant = 0.0
    v_avg = 0.0
    n_avg = 1
    dfTracks = df.loc[df['oid']==oid][['secondsfromstart', 'speed']].sort_values(by=['secondsfromstart']).reset_index()
    for index, row in dfTracks.iterrows():
      if row['speed']>0.0:#Eliminamos velocidades 0
        diff_t = row['secondsfromstart']-seconds_ant
        s+= row['speed']*diff_t
        t+= diff_t
        v_avg+=row['speed'] #v_avg_n = ((n-1)*v_avg_n-1 + v)/n
        n_avg+=1
      seconds_ant = row['secondsfromstart']
    if t==0.0: #Es posible que no haya tracks
      v = 3000.0 #velocidad absurda para tenerlos identificados
      v_avg = 3000.0 #velocidad absurda para tenerlos identificados
    else:
      v = s/t
      v_avg = v_avg/(n_avg-1)
    df = pd.concat([df, pd.DataFrame({
      'oid': [oid],
      'travel_time_track': [t], 
      'avg_speed_track': [v], 
      'avg_speed': [v_avg]
    })])
  print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'), 'Fin')
  # Donde no haya valor de speed (3000), ponemos el valor medio.
  vars_mean = {'avg_track_speed':3000.0, 'avg_speed':3000.0, 'travel_time_track':0.0}
  query = ' & '.join([f'{k}!={v}' for k, v in vars_mean.items()])
  dfAux = dfSpeeds.query(query)
  dfAux = dfAux[[k for k, v in vars_mean.items()]].mean().reset_index()
  for k, v in vars_mean.items():
    dfSpeeds[k] = pd.np.where(
      dfSpeeds[k]==v, dfAux.loc[dfAux['index']==k, 0], 
      dfSpeeds[k]
    )
  return df

'''
  Funcion que añade variables extras como Fin de Semana, Horario, etc
  Recibe el Dataframe de Viajes
  Devuelve tantas variables como se hayan definido:
    day_of_week => Literal dia de la semana.
    weekend => Si el día es Fin de semana (weekend) o laboral (working day)
    hour_of_day => Hora del día (0-23)
    hour_type => Agrupamiento de horas en el día.
    trip_type => Agrupamiento de las estaciones de enganche y desenganche:
      Misma Estación, Retiro (origen y destino es en Retio), a y desde Retiro.
    
'''
def addFeatures(df):
  def getHourType(hour):
    if hour>=23:
      return "(23:00-07:00)"
    if hour<8:
      return "(23:00-07:00)"
    if hour<13:
      return "(07:00-12:00)"
    if hour<19:
      return "(12:00-18:00)"
    if hour<23:
      return "(18:00-23:00)"
    
  def getTripType(id_unplug, id_plug):
    if id_unplug==id_plug:
      return "Same Station"
    if id_unplug in retiro_stations:
      if id_plug in retiro_stations:
        return "Retiro"
      return "From Retiro"
    return "To Retiro"
     
  df['daysample'] = pd.to_datetime(df['daysample'])
  df['day_of_week'] = df['daysample'].dt.dayofweek
  # 5 => Sat, 6 =>Sun, 0 => Mon, 1 => Tue...
  df['weekend'] = pd.np.where(
      df['day_of_week']>4, 'Weekend', 'Laboralday'
  )
  
  df['datesample'] = pd.to_datetime(df['datesample'])
  df['hour_of_day'] = df['datesample'].dt.hour
  df['day_of_week'] = '(' + df['daysample'].dt.dayofweek.astype('str') 
  df['day_of_week']+= ') ' + df['daysample'].dt.day_name()  
  df['hour_type'] = [getHourType(hour) for hour in df['hour_of_day']]
  df['trip_type'] = [getTripType(id_unplug, id_plug) 
    for id_unplug, id_plug in 
      zip(df['idunplug_station'], df['idplug_station'])
  ]
  return df
'''
  Función que devuelve un código que identifica al viaje 
  basado en un pool de variable.
  Recibe el DataFrame de Viajes y las variables
  Devuelve el Dataframe con la variable 'code'
'''
def getCode(df, vars):
  dfAux = pd.DataFrame({'code':['' for i in range(df.shape[0])]})
  for var in vars:
    if var in ['weekend', 'trip_type', 'desc_user_type']:
      dfAux['code']+=df[var].str.slice(0, 1)
    elif var=='hour_type':
      dfAux['code']+=df[var].str.slice(1, 3)
    elif var=='desc_ageRange':
      dfAux['code']+=df[var].str.slice(0, 2)
    elif var=='day_of_week':
      dfAux['code']+=df[var].str.slice(4, 2)
  return dfAux['code']

'''
  Función que identifica los outliers por columna
  Copia pega de la vista en clase con Rafa.
'''
def identificar_outliers(df, col_name):
  #q1, q3 = np.percentile(df[col_name], [25, 75])
  q1, q3 = np.quantile(df[col_name], [0.25, 0.75])
  step = 1.5*(q3-q1)
  mask = df[col_name].between(q1 - step, q3 + step, inclusive=True) #identifica los que estan dentro
  iqr = df.loc[~mask].index #negado de la mascara que hemos aplicado.
  return list(iqr)

'''
  Función que Calcula y devuelve el conjunto potencia de la lista c.
'''
def combinatoria(c):
  if len(c) == 0:
    return [[]]
  r = combinatoria(c[:-1]) 
  r = r + [s + [c[-1]] for s in r]
  return [e for e in sorted(r, key=lambda s: (len(s), s))]

'''
  Función que Calcula el coeficiente de cramer_v.
'''
def cramers_v(confusion_matrix):
  import scipy.stats as sc
  chi2 = sc.chi2_contingency(confusion_matrix)[0]
  n = confusion_matrix.sum().sum()
  phi2 = chi2/n
  r,k = confusion_matrix.shape
  phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
  rcorr = r-((r-1)**2)/(n-1)
  kcorr = k-((k-1)**2)/(n-1)
  return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

'''
  Función que genera relación entre nodos con el p_value utilizando ks_2samp
  Para cada tupla de valores proporcionados por las variables combi_var
  calcula si la distribución de cada variable en vars es igual.
'''
def generateRelationPValue(combi_var, vars, df):
  x_vars = {}
  for var in vars:
    x_vars[var] = pd.DataFrame(columns=['nodeA', 'nodeB', 'p_' + var])
  values = sorted(df['code'].unique().tolist())
  for x_var in x_vars:
    for i in range(len(values)):
      data1=dfTripsIQR.loc[dfTripsIQR['code']==values[i]][x_var]
      for j in range(i+1, len(values)):
        data2=dfTripsIQR.loc[dfTripsIQR['code']==values[j]][x_var]
        t_stat, p_value = stats.ks_2samp(data1, data2)
        #print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'), x_var, values[i], values[j], t_stat, p_value)
        p_var = 'p_' + x_var
        x_vars[x_var] = pd.concat([x_vars[x_var], pd.DataFrame({
          'nodeA': [values[i]],
          'nodeB': [values[j]],
          p_var: [p_value]       
        })])
  dfNodes = pd.DataFrame()
  for x_var in x_vars:
    if dfNodes.empty:
      dfNodes = x_vars[x_var]
    else:
      dfNodes = dfNodes.merge(x_vars[x_var], 
        left_on=['nodeA', 'nodeB'],
        right_on=['nodeA', 'nodeB'],
        how='inner'
      )
  return dfNodes

'''
  Función que genera la tupla necesaria para renderizar un HeatMap
  de importacion de variables sobre un cluster
  df => dataframe junto con una variable que indica el cluster al que pertenece.
  vars => variables implicadas
  cluster => listado de clusters
  cluster_var => el nombre de la variable del dataframe que tiene 
  el valor del cluster
'''
def generateClusterMap(df, vars, clusters, cluster_var):
  total_registers = df.shape[0]
  dfWeightGlobal = pd.DataFrame(columns=['label', 'weight_global'])
  for var in vars:
    dfAmount = df.groupby(var)[var].count().to_frame()
    dfAmount.columns = ['weight_global']
    dfAmount['label'] = [var + '_' + value for value in dfAmount.index]
    dfAmount['weight_global'] = dfAmount['weight_global']/total_registers
    dfAmount = dfAmount.reset_index()
    dfAmount = dfAmount.drop(columns=[var])
    dfWeightGlobal = pd.concat([dfWeightGlobal, dfAmount], sort=True)
  dfWeightGlobal = dfWeightGlobal.sort_values('label')
  z = []
  for cluster in clusters:
    dfCluster = df.loc[df[cluster_var]==cluster][vars]
    total_registers = dfCluster.shape[0]
    dfWeightCluster = pd.DataFrame(columns=['label', 'weight_cluster'])
    for var in vars:
      dfAmount = dfCluster.groupby(var)[var].count().to_frame()
      dfAmount.columns = ['weight_cluster']
      dfAmount['label'] = [var + '_' + value for value in dfAmount.index]
      dfAmount['weight_cluster'] = dfAmount['weight_cluster']/total_registers
      dfAmount = dfAmount.reset_index()
      dfAmount = dfAmount.drop(columns=[var])
      dfWeightCluster = pd.concat([dfWeightCluster, dfAmount], sort=True)
    dfWeightCluster = dfWeightCluster.merge(
      dfWeightGlobal, 
      left_on='label', 
      right_on='label', 
      how='outer', 
      indicator=True
    )
    dfWeightCluster['weight_cluster'] = pd.np.where(
      dfWeightCluster['_merge']=='both', 
      dfWeightCluster['weight_cluster'], 
      0.0
    )
    dfWeightCluster = dfWeightCluster.drop(columns=['_merge'])
    dfWeightCluster = dfWeightCluster.sort_values('label')
    #z.append((dfWeightCluster['weight_cluster']*dfWeightCluster['weight_global']).to_list())
    z.append(dfWeightCluster['weight_cluster'].to_list())
  x = dfWeightGlobal['label'].to_list()
  y = ['cluster: ' + str(cluster) for cluster in clusters]
  return x, y, z

## Funciones para graficar

In [0]:
#Inventario de funciones para Graficar => Plotly
colors=['#09BB9F','#15607A','#FFD7E9','#1D81A2','#EB89B5','#18A1CD','#879DC6']

# Método para que funcione Plotly en colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  from plotly import tools
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  tools.set_credentials_file(
        username='jazzphoenix', 
        api_key='euOGc1f62ScWry0fPG8D'
  )
  init_notebook_mode(connected=True)

#Método para generar histogramas de forma dinámica
def generateHistogram(filter_var, x_var, df, cols):
  from plotly.offline import iplot
  import plotly.graph_objs as go
  from plotly import tools

  enable_plotly_in_cell()
  uniq_datas = sorted(df[filter_var['name']].unique().tolist())
  rows = len(uniq_datas) // cols + len(uniq_datas) % cols
  fig = tools.make_subplots(rows=rows, cols=cols, print_grid=False)
  i = 1
  j = 1
  k = 1
  for uniq_data in uniq_datas:
    #histnorm='percent', 'probability'
    trace = go.Histogram(
      x = df.loc[df[filter_var['name']]==uniq_data, x_var],
      name = filter_var['desc'] + ' ' + uniq_data,
      marker = dict(
        color=colors[i-1],
        line = dict(width = 0.5, color = "black")
      ),
      opacity=0.75,
    )
    fig.append_trace(trace, j, k)
    '''
    trace2 = go.Histogram(
      x = df.loc[df[filter_var['name']]==uniq_data, x_var]/2,
      name = filter_var['desc'] + ' 2 ' + uniq_data,
      marker = dict(
        color=colors[i],
        line = dict(width = 0.5, color = "black")
      ),
      opacity=0.75,
    )
    fig.append_trace(trace2, j, k)
    '''
    fig['layout']['xaxis' + str(i)].update(title=x_var)
    fig['layout']['yaxis' + str(i)].update(title='total users')
    i+=1
    if k==cols:
      k = 0
      j+= 1
    k+=1

  fig['layout'].update(showlegend=True, 
    title='Histogram ' + x_var + ' by ' + filter_var['desc'],
    height = 400*rows,
  )
  iplot(fig, filename='histogram_'+ x_var + '_' + filter_var['desc'])
  
#Método para generar un quesito de forma dinámica
def generatePie(x_var, filter_var, df):
  from plotly.offline import iplot
  import plotly.graph_objs as go

  enable_plotly_in_cell()
           
  filter_values = ['Total register']
  if filter_var!='':
    filter_values+= df[filter_var].unique().tolist()
  labels = sorted(df[x_var].unique().tolist())
  i=0
  annotations = []
  data = []
  for filter_value in filter_values:
    values = []
    for label in labels:
      if filter_value=='Total register':
        values.append(df.loc[df[x_var]==label].shape[0])
      else:
        values.append(df.loc[(df[x_var]==label) & (df[filter_var]==filter_value)].shape[0])
    trace = {
      'labels': labels,
      'values': values,
      'type': 'pie',
      'name': x_var,
      'hole': 0.4,
      'domain': {'column': i}
    }
    data.append(trace)
    annotation = {
      "font": {"size": 16},
      "showarrow": False,
      "text": filter_value,
      "x": 0.11 + 1.2*i/len(filter_values),
      "y": 0.5
    }
    annotations.append(annotation)
    i+=1
    fig = {
      'data': data, 
      'layout': {
        'title': x_var,
        "grid": {"rows": 1, "columns": len(filter_values)},
        "annotations": annotations
    }}
  iplot(fig, filename='pie_' + x_var + '_' + filter_var)

#Método para generar gráficos BoxPlot de forma dinámica
def generateBoxPlot(x_var, filter_var, df, title):
  from plotly.offline import iplot
  import plotly.graph_objs as go

  enable_plotly_in_cell()
  values = df[filter_var].unique().tolist()
  data = []
  for value in values:
    #Añadimos un caracter para que no lo transforme a número
    trace = go.Box(
      y=df.loc[df[filter_var]==value][x_var],
      name='·' + str(value),
      notched=True,
      boxmean='sd'
    )
    data.append(trace)
  layout = go.Layout(
    title = title
  )
  fig = go.Figure(data=data,layout=layout)    
  iplot(fig)

#Método para generar Scatter Matrix de forma dinámica 
def generateScatterMatrix(vars, df):
  from plotly.offline import iplot
  import plotly.graph_objs as go
  from plotly import tools
  import plotly.figure_factory as ff

  enable_plotly_in_cell()
  fig = tools.make_subplots(rows=len(vars), cols=len(vars), print_grid=False)
  k=1
  for i in range(len(vars)):
    for j in range(len(vars)):
      varx = vars[i]
      vary = vars[j]
      if varx==vary:
        # Create distplot with curve_type set to 'normal'
        fig_plot = ff.create_distplot(
            [df[varx]], 
            [varx], 
            bin_size=.5, 
            colors=['#15607A']
        )
        dist_plot=fig_plot['data']        
        fig.append_trace(dist_plot[0], i+1, j+1)
        fig.append_trace(dist_plot[1], i+1, j+1)
      else:
        trace = go.Scattergl(
          x = df[varx],
          y = df[vary],
          mode = 'markers',
          marker = dict(line = dict(width = 1, color = '#15607A'))
        )
        fig.append_trace(trace, i+1, j+1)
      fig['layout']['xaxis' + str(k)].update(title=varx)
      if varx!=vary:
        fig['layout']['yaxis' + str(k)].update(title=vary)
      k+=1
  fig.layout.update(showlegend=False, 
    title='Scatter Matrix Variables Numéricas',
    height = 300*len(vars), width=1000
  )

  iplot(fig, filename='scatter_matrix')  

#Método para generar HeatMap
def generateHeatMap(x, y, z, title, colorscale, colorbar):
  from plotly.offline import iplot
  import plotly.graph_objs as go

  enable_plotly_in_cell()
  
  trace = go.Heatmap(
      z=z,
      x=x,
      y=y,
      xgap = 3,
      ygap = 3,
      colorscale = colorscale,
      colorbar = colorbar
  )
  data=[trace]
  layout = go.Layout(
    title = title,     
    margin=go.layout.Margin(l=100, b=200),
    yaxis={'tickangle':-45}
    
  )
  fig = go.Figure(data=data, layout=layout)
  iplot(fig, filename='labelled-heatmap') 
  
  
#Método para pintar la relación de correlación entre dos variables 
def generateCrosstabBars(crosstab, xvar, yvar, percentage):
  from plotly.offline import iplot
  import plotly.graph_objs as go
  from plotly import tools
  enable_plotly_in_cell()
  
  fig = tools.make_subplots(rows=1, cols=2, print_grid=False)
  data = []
  x = crosstab.index.tolist()
  if percentage:
    columnsPercertage = []
    for i in range(len(x)):
      columnsPercertage.append(sum(crosstab.iloc[i, :].tolist()))
  for i in range (crosstab.columns.shape[0]):
    if percentage:
      a = crosstab.iloc[:, i].tolist()
      b = columnsPercertage
      c = [str(round(100*x/y,2))+'%' for x, y in zip(a, b)]
    else:
      c = crosstab.iloc[:, i].tolist()
    trace = go.Bar(
      x=x,
      y=c,
      name=str(crosstab.columns[i])
    )
    data.append(trace)

  layout = go.Layout(
      barmode='group',
      title = 'CrossTab ' + xvar + '<->' + yvar,
      xaxis = dict(title = xvar),
      yaxis = dict(title = yvar)
  )

  fig = go.Figure(data=data, layout=layout)
  iplot(fig, filename='grouped-bar')

def generateLineChar(x, y, names, title, xtitle, ytitle):
  from plotly.offline import iplot
  import plotly.graph_objs as go
  enable_plotly_in_cell()
  data = []
  for i in range(len(x)):
    if x[i]:
      trace = go.Scatter(x = x[i], y = y[i], name = names[i])
    else:
      trace = go.Scatter(y = y[i], name = names[i])
    data.append(trace)
  layout = dict(title = title,
    xaxis = dict(title = xtitle),
    yaxis = dict(title = ytitle),
  )
  fig = dict(data=data, layout=layout)
  iplot(fig, filename='basic-line')  

#Carga de Dataset
A partir del módulo 3 y 4, se disponía de contenido tanto en HDFS como en Hive de todos los registros relacionados con los datos de Uso de las Bicicletas.

Por problemas de espacio, ya se ha hecho una labor previa de filtrado con las estaciones pertenecientes al retiro.

Se contemplan dos escenarios:


*   Estaciones que está alrededor del Parque Retiro (201808_Usage_Bicimad.json.square.filter).
*   Estaciones que pertenecen al Distrito Retiro (201808_Usage_Bicimad.json.district.filter)

Incluiremos, además un campo **weekend** que identifique si el día corresponde a Día Laboral (0) o Fin de Semana (1).

Declararemos dataFrames auxiliares para disponer de descripciones de ciertas variables categóricas: **user_type, ageRange, idplug_station, idunplug_station**.
La información de las estaciones se ha capturado usando la API BiciMad (módulo 3 y 4) y generando un **station.csv** para su carga. Ese fichero contiene el id de la estación, nombre, total_bases así como latitud y longitud.

In [23]:
# Relación de variables categóricas user_type y ageRange
dfUserType = pd.DataFrame({
    'id_user_type': [0, 1, 2, 3], 
    'desc_user_type': ['Unknown', 'Annual', 'Occasional', 'Company']
})
dfAgeRange = pd.DataFrame({
    'id_ageRange': [0, 1, 2, 3, 4, 5, 6], 
    'desc_ageRange': [
        'Unknown', 
        '00-16 years', 
        '17-18 years', 
        '19-26 years', 
        '27-40 years', 
        '41-65 years', 
        '66- years']
})
# Relación de id de Estaciones 
dfStations = pd.read_csv('stations.csv', sep=';')
dfStations.head(5)

,id,latitude,longitude,name,number,address,total_bases
0,1,40.416896,-3.702425,Puerta del Sol A,1a,Puerta del Sol nº 1,24
1,2,40.417001,-3.702421,Puerta del Sol B,1b,Puerta del Sol nº 1,24
2,3,40.420589,-3.705842,Miguel Moya,2,Calle Miguel Moya nº 1,24
3,4,40.430294,-3.706917,Plaza Conde Suchil,3,Plaza del Conde Suchil nº 2-4,18
4,5,40.428552,-3.702587,Malasaña,4,Calle Manuela Malasaña nº 5,24


In [24]:

  
dfTrips = pd.read_csv(file + '.filter', sep=';', dtype={
    'datesample':'str', 
    'oid':'str', 
    'user_day_code':'str', 
    'idplug_base':'int64', 
    'user_type':'int64', 
    'idunplug_base':'int64', 
    'travel_time':'int64', 
    'idunplug_station':'int64', 
    'ageRange':'int64', 
    'idplug_station':'int64', 
    'zip_code':'str', 
    'longitude':'float64', 
    'latitude':'float64', 
    'var':'str', 
    'speed':'float64', 
    'secondsfromstart':'float64', 
    'daysample':'str'
})


'''
Filtrado de los viajes de las estaciones de retiro

dfTrips = dfTrips.loc[
  (dfTrips['idunplug_station'].isin(retiro_stations)) & 
  (dfTrips['idunplug_station'].isin(filter_square)
)].reset_index()
'''

#dfSpeeds = calculateSpeed(dfTrips)
#Usamos fichero de Speeds precalculado para optimizar tiempo de procesado.
dfSpeeds = pd.read_csv(file + '.speed', sep=';')
dfSpeeds['travel_space_avg'] = dfSpeeds['avg_speed']*dfSpeeds['travel_time_track']
dfSpeeds['travel_space_track'] = dfSpeeds['avg_track_speed']*dfSpeeds['travel_time_track']
#Eliminamos los Tracks y nos quedamos solo con informacion de viaje
dfTrips.drop([
		'idplug_base', 
		'idunplug_base', 
		'zip_code', 
		'longitude', 
		'latitude', 
		'var', 
		'speed', 
		'secondsfromstart'
	], axis=1, inplace=True)
dfTrips.drop_duplicates(inplace=True)

#Añadimos información de velocidades
dfTrips = dfTrips.merge(dfSpeeds, left_on='oid', right_on='oid', how='inner')

#Añadimos descripcion de user_type
dfTrips = dfTrips.merge(dfUserType, 
  left_on=['user_type'], right_on=['id_user_type'],
  how='inner'
)

#Añadimos descripcion de ageRange
dfTrips = dfTrips.merge(dfAgeRange, 
  left_on=['ageRange'], right_on=['id_ageRange'],
  how='inner'
)

#Añadimos informacion de estación de enganche
dfTrips = dfTrips.merge(dfStations, 
  left_on=['idplug_station'], right_on=['id'],
  how='inner'
)
dfTrips.rename(columns={
    'latitude': 'plug_station_latitude', 
    'longitude': 'plug_station_longitude',
    'name': 'plug_station_name',
    'number': 'plug_station_number',
    'address': 'plug_station_address',
    'total_bases': 'plug_station_total_bases'
}, inplace=True)

#Añadimos informacion de estación de des enganche
dfTrips = dfTrips.merge(dfStations, 
  left_on=['idunplug_station'], right_on=['id'],
  how='inner'
)
dfTrips.rename(columns={
    'latitude': 'unplug_station_latitude', 
    'longitude': 'unplug_station_longitude',
    'name': 'unplug_station_name',
    'number': 'unplug_station_number',
    'address': 'unplug_station_address',
    'total_bases': 'unplug_station_total_bases'
}, inplace=True)
dfTrips.drop(['ageRange', 'user_type', 'id_x', 'id_y'], axis=1, inplace=True)

dfTrips = addFeatures(dfTrips)

dfTrips.head()




,datesample,oid,user_day_code,travel_time,idunplug_station,idplug_station,daysample,avg_speed,avg_track_speed,travel_time_track,travel_space_avg,travel_space_track,id_user_type,desc_user_type,id_ageRange,desc_ageRange,plug_station_latitude,plug_station_longitude,plug_station_name,plug_station_number,plug_station_address,plug_station_total_bases,unplug_station_latitude,unplug_station_longitude,unplug_station_name,unplug_station_number,unplug_station_address,unplug_station_total_bases,day_of_week,weekend,hour_of_day,hour_type,trip_type
0,2018-08-01 01:00:00,5b6779012f384302541d6821,0279839c9c173d430f87eaff66719268979afc65dde413...,498,90,35,2018-08-01,6.786667,6.660588,391.0,2653.586667,2604.29,1,Annual,0,Unknown,40.416364,-3.706897,Calle Mayor,31,Calle Mayor nº 20,27,40.421501,-3.680008,Puerta de Madrid,85,Avenida de Menéndez Pelayo nº 11,27,(2) Wednesday,Laboralday,1,(23:00-07:00),From Retiro
1,2018-08-02 01:00:00,5b68ca8a2f38433e00dde3d1,9fc38469f13545724daa122e2a6aaea2a358d9eb8fe739...,446,90,35,2018-08-02,6.395000,6.413531,371.0,2372.545000,2379.42,1,Annual,0,Unknown,40.416364,-3.706897,Calle Mayor,31,Calle Mayor nº 20,27,40.421501,-3.680008,Puerta de Madrid,85,Avenida de Menéndez Pelayo nº 11,27,(3) Thursday,Laboralday,1,(23:00-07:00),From Retiro
2,2018-08-03 00:00:00,5b6a1c1b2f38434d8862b439,e21bb91b9359de1b581f92c44232671f634a47b13c4952...,454,90,35,2018-08-03,5.648750,5.749933,446.0,2519.342500,2564.47,1,Annual,0,Unknown,40.416364,-3.706897,Calle Mayor,31,Calle Mayor nº 20,27,40.421501,-3.680008,Puerta de Madrid,85,Avenida de Menéndez Pelayo nº 11,27,(4) Friday,Laboralday,0,(23:00-07:00),From Retiro
3,2018-08-09 00:00:00,5b72053c2f3843505c6bcad8,52bb7ba091a95094e4e75daa9dd1ca0ac1bdaf9d7711af...,512,90,35,2018-08-09,4.625000,5.130247,446.0,2062.750000,2288.09,1,Annual,0,Unknown,40.416364,-3.706897,Calle Mayor,31,Calle Mayor nº 20,27,40.421501,-3.680008,Puerta de Madrid,85,Avenida de Menéndez Pelayo nº 11,27,(3) Thursday,Laboralday,0,(23:00-07:00),From Retiro
4,2018-08-10 00:00:00,5b7356562f38434438ae69b0,9ddb502e86f00d70eaf8e21c5a9317e2882944fb684a2b...,436,90,35,2018-08-10,5.834286,5.791040,404.0,2357.051429,2339.58,1,Annual,0,Unknown,40.416364,-3.706897,Calle Mayor,31,Calle Mayor nº 20,27,40.421501,-3.680008,Puerta de Madrid,85,Avenida de Menéndez Pelayo nº 11,27,(4) Friday,Laboralday,0,(23:00-07:00),From Retiro


##Usuarios
Parece Interesante agrupar todos los viajes del dataset para hacer un estudio desde el punto de vista de usuarios, por ejemplo, cuántos viajes realizan o tiempo medio de los viajes. Destacar que la información que se puede agregar de usuarios es por día; variable user_day_code.


In [25]:
user_vars = [
    'user_day_code', 
    'id_user_type', 
    'desc_user_type',
    'id_ageRange',
    'desc_ageRange',
    'day_of_week',
    'weekend'
]
dfUsers = dfTrips[['oid','travel_time'] + user_vars].drop_duplicates()

dfUsers = dfUsers.groupby(user_vars).agg({
					'oid':[('total_trips', 'count')],
					'travel_time':[('avg_travel_time', 'mean')]
}).reset_index()
dfUsers.columns = [b if b!="" else a for a, b in dfUsers.columns]

dfUsers.head()



,user_day_code,id_user_type,desc_user_type,id_ageRange,desc_ageRange,day_of_week,weekend,total_trips,avg_travel_time
0,00012269e83c8c50811864fab4470b7b58c8e5ae349277...,1,Annual,0,Unknown,(1) Tuesday,Laboralday,2,1112.5
1,0004bb097cf07e2b24d6ea3803d439c737b174a36b65e9...,1,Annual,0,Unknown,(3) Thursday,Laboralday,1,256.0
2,000b85f59b35902d53017737731b39bd8cbf6f1588006a...,1,Annual,0,Unknown,(4) Friday,Laboralday,1,513.0
3,00134c8ae3550375ee328cddd642f1b080466c5e21e484...,1,Annual,0,Unknown,(1) Tuesday,Laboralday,1,890.0
4,001baafc9de66d58b27765f5590065982c79ef85ef53e5...,1,Annual,5,41-65 years,(5) Saturday,Weekend,1,3405.0


#Análisis Exploratorio
Utilizaremos las funciones propias de Pandas info y describe para hacer un primer análisis. Destacar que aunque las variables son numéricas, realmente, muchas de ellas deberían considerarse como categóricas;por ejemplo: **user_type,  ageRange, idplug_station e idunplug_station**.


##Usuarios

A continuación se va a analizar el dataset haciendo foco en el usuario, para ver si tiene sentido estudiar el comportamiento teniendo como centro o no. 
En resumen que se ve abajo, es posible ya vislumbrar que la gran mayoría de tipos de usuario son Anuales, como indica que en el tercer cuartil predomina el tipo 1. Además otra pista que puede llegar a dar este resumen es que la mayoria de usuarios hacen un único viaje al día, como lo indica el tercer cuartil. Incluso en la media de tiempo por viaje del usuario se puede ver la diferencia entre el tercer cuartil y el maximo, siendo indicador de posibles outliers.

In [0]:
dfUsers.describe()

,id_user_type,id_ageRange,total_trips,avg_travel_time
count,12236.000000,12236.000000,12236.000000,12236.000000
mean,1.098071,2.307453,1.212161,1281.838753
std,0.351343,2.171468,1.247187,5519.681684
min,1.000000,0.000000,1.000000,1.000000
25%,1.000000,0.000000,1.000000,458.000000
50%,1.000000,3.000000,1.000000,681.000000
75%,1.000000,4.000000,1.000000,1116.000000
max,3.000000,6.000000,32.000000,481814.000000


In [0]:
dfUsers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12236 entries, 0 to 12235
Data columns (total 9 columns):
user_day_code      12236 non-null object
id_user_type       12236 non-null int64
desc_user_type     12236 non-null object
id_ageRange        12236 non-null int64
desc_ageRange      12236 non-null object
day_of_week        12236 non-null object
weekend            12236 non-null object
total_trips        12236 non-null int64
avg_travel_time    12236 non-null float64
dtypes: float64(1), int64(3), object(5)
memory usage: 860.4+ KB


### Distribución total_trips y user_type

In [26]:
filter_var = {'name':'desc_user_type', 'desc': 'User Type'}
x_var = 'total_trips'
cols = 3

generateHistogram(filter_var, x_var, dfUsers, cols)

Con el visual de los histogramas de frecuencia por tipo de usuario, queda claro lo que se empezaba a entender en el resumen, la gran mayoria de usuarios hacen un único viaje. Empieza a entenderse por los puntos más altós en cada tipo de usuario que, según su modelo de contratación del servicio, predomina puede ser el Anual por encima del ocasional y el de empresa.

Detalles interesantes:
- Tipo Anual y Ocasional hay 1,2,3 viajes y su mayoría predomina 1 viaje (10700 users)
- Empresa hay hasta 14 grupos de viajes predomina 1 viaje, y rango de 2 a 10 viajes por usuario


### Distribución por total_trips y ageRange

In [27]:
filter_var = {'name':'desc_ageRange', 'desc': 'Age Range'}
x_var = 'total_trips'
cols = 2
  
generateHistogram(filter_var, x_var, dfUsers, cols)

Si se cambia el tipo de usuario por el rango de edad ocurre exactamente igual, no hay rango de edad en la que no predomine de manera importante un solo viaje por día. De este modo puede a confirmarse que mayoritariamente en el dataset de agosto de 2018 en la zona del Parque del Retiro los usuarios que hicieron uso del servicio de Bicimad fue un único viaje por día. 

Datos de interes:
- Hay 7 tramos de edad (Unknown, <16,17-18,19-26,27-40,41-65,>66)
- Los tramos con más volumen después del Unknown es de 27-40, seguido de 41-65

A continuación vamos a ver de modo más visual el comportamiento de los tipos de usuario y rango de edad. Ya que en el primero se empezaba a ver un predomino importante de uno de los tipos de abono.

### Pie ageRange por weekend

In [28]:
x_var = 'desc_ageRange'
filter_var = 'weekend'
generatePie(x_var, filter_var, dfUsers)


Con los siguientes gráficos de tartas puede entenderse como un gran desconocimiento del dato de rango de edad del dataset, 45% de Unknown, que puede llegar a afectar a la hora de clusterizar. Además se ve como grupos que mayor uso hacen de Bicimad los que estan entre 27-40 como principales y secundarios 41-65. El resto de rangos se ve un  uso menor del servicio. 

Datos de interes:
- El uso es mayor en laboral que fin de semana para todos los tramos de edad.
- La distribución de tramos de edad y uso según el tipo de día es equivalente aparentemente en proporción tanto para laboral como finde, la tendencia es que el uso decrementa en fin de semana para la mayoría los tramos de edad.
- Sólo se aprecia un ligero incremento en fin de semana en los tramos 17-18 y >66





### Pie user_type por weekend

In [29]:
x_var = 'desc_user_type'
filter_var = 'weekend'
generatePie(x_var, filter_var, dfUsers)

Confirmamos el uso mayoritario del tipo de usuario Anual frente a los otros dos tipos de abono. 

Datos de interés:
- La tendencia general es que se usa más en laboral que en finde para los tipos Anual y Company
- Por el contrario el ocasional es usado más el fin de semana, doblando en proporción. 

##Viajes

Tras lo visto en los datos con foco en los usuarios, se ve claro y necesario cambiar el foco a los viajes y analizar los mismos por tipo de uso y caracteristicas de usuarios para poder entender el uso y sus diferentes agrupamientos.

In [0]:
dfTrips.describe()

,travel_time,idunplug_station,idplug_station,avg_speed,avg_track_speed,travel_time_track,travel_space_avg,travel_space_track,id_user_type,id_ageRange,plug_station_latitude,plug_station_longitude,plug_station_total_bases,unplug_station_latitude,unplug_station_longitude,unplug_station_total_bases,hour_of_day
count,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000
mean,1411.972492,77.745078,83.009574,4.632411,4.628131,1061.046114,4701.416302,4560.004886,1.277508,2.302117,40.418901,-3.691085,24.058994,40.416009,-3.682386,25.017597,14.157834
std,5173.290281,11.104111,44.268030,1.470961,1.541871,1255.192657,6607.804269,5650.500870,0.647697,2.169340,0.011680,0.012860,1.739516,0.005117,0.004769,1.420360,6.007084
min,1.000000,64.000000,1.000000,0.300000,0.300000,14.000000,7.700000,7.700000,1.000000,0.000000,40.391939,-3.724653,12.000000,40.408280,-3.688822,24.000000,0.000000
25%,458.000000,73.000000,53.000000,3.778874,3.727735,419.000000,1926.787500,1937.160000,1.000000,0.000000,40.409808,-3.701569,24.000000,40.409808,-3.688398,24.000000,10.000000
50%,714.000000,78.000000,80.000000,4.632411,4.628131,720.000000,3171.250721,3161.105000,1.000000,3.000000,40.419210,-3.690965,24.000000,40.419752,-3.680008,24.000000,15.000000
75%,1299.500000,90.000000,111.000000,5.300192,5.326808,1061.046114,4915.201605,4910.660301,1.000000,4.000000,40.425394,-3.680008,24.000000,40.421180,-3.678484,27.000000,19.000000
max,481814.000000,107.000000,175.000000,30.720000,30.720000,29480.000000,190913.256696,106831.670000,3.000000,6.000000,40.459235,-3.668909,30.000000,40.421501,-3.676681,27.000000,23.000000


In [0]:
dfTrips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14832 entries, 0 to 14831
Data columns (total 33 columns):
datesample                    14832 non-null datetime64[ns]
oid                           14832 non-null object
user_day_code                 14832 non-null object
travel_time                   14832 non-null int64
idunplug_station              14832 non-null int64
idplug_station                14832 non-null int64
daysample                     14832 non-null datetime64[ns]
avg_speed                     14832 non-null float64
avg_track_speed               14832 non-null float64
travel_time_track             14832 non-null float64
travel_space_avg              14832 non-null float64
travel_space_track            14832 non-null float64
id_user_type                  14832 non-null int64
desc_user_type                14832 non-null object
id_ageRange                   14832 non-null int64
desc_ageRange                 14832 non-null object
plug_station_latitude         14832 non-nul

En este primer resumen, puede enterse que la media de uso se situa en una franja horaria de mediodia. La velocidad media del viaje indica posibles outliers y un comportamiento muy parejo con la velocidad media del track. Del mismo modo ocurre con las medidas de tiempo medio del viaje y tiempo medio del track. De este modo se entiende que lo más probable con una variable por cada sería suficiente. Para confirmar esta premisa visualizaremos la matriz de correlaciones a continuación.

###Distribución del dataset e indentificación de outliers

In [0]:
generateScatterMatrix([
  'travel_time', 
  'avg_speed', 
  'avg_track_speed', 
  'travel_time_track'
], dfTrips)



Output hidden; open in https://colab.research.google.com to view.

Como puede entenderse la premisa que se mencionaba sobre la relacion entre ambas variables de velocidad como ambas de tiempo estan correladas. La variable velocidad tiene una correlación muy alta, lo que indica que dichas variables muy similares casi iguales en la información que aportan. Por lo que con una de ellas ya sería suficiente. En el caso del tiempo medio no hay una correlación tan alta, pero si indica de modo que problable que en este caso también con una de ambas variables sea suficiente.

#### Matriz de correlación de variables numéricas

Para confirmar las correlaciones que se ven en los anteriores gráficos, nos disponemos a ejecutar la matriz de correlaciones.



In [0]:
dfCorr = dfTrips[[
    'travel_time', 
    'avg_speed', 
    'avg_track_speed', 
    'travel_time_track'
]].corr(method='pearson')

z = []
for index, row in dfCorr.iterrows():
  z.append(row)
    
generateHeatMap(
    dfCorr.index, 
    dfCorr.index, 
    z, 
    'Correlación Variables Numéricas',
    None,
    None
)    



Finalmente se confirman las correlaciones anteriormente mencionadas. Por lo que se preservarán en este analisis una varíable por cada par, "travel_time" y "avg_speed"

### Eliminación de los outliers mediante el metodo IQR

In [0]:
columns = ['travel_time', 'avg_speed']
#Transformación a logaritmo para mejorar la distribución normal.
for col in columns:
  dfTrips[col] = np.log(dfTrips[col])

outliers_index = []
for col in columns:
  outliers_index+= identificar_outliers(dfTrips, col)
dfTripsIQR = dfTrips.drop(outliers_index).reset_index(drop = True)


In [0]:
generateScatterMatrix([
  'travel_time', 
  'avg_speed', 
  'avg_track_speed', 
  'travel_time_track'
], dfTripsIQR)

Output hidden; open in https://colab.research.google.com to view.

Tras la eliminación de los outliers puede observarse como la distribución de las variables se vuelve más Gausiana y los datos ya están mucho mas compactados para poder modelizarlos.

###BoxPlot Code Group

Crearemos una variable "code" con la combinatoria de valores de las distintas variables categóricas que queremos analizar juntas. La idea es analizar las distribuciones de las variables numéricas **travel_time** y **avg_speed** para cada valor de la combinatoria de las variables catégoricas y ver si existen similitudes. Para analizar dichas similitudes, utilizaremos **Kolmogorov-Smirnov** ya que T-Test está más orientado a distribuciones normales. Partiremos del análisis de una única variable e iremos iterando añadiendo cada vez una variable más hasta llegar a todas las variables.

In [0]:
from scipy import stats

vars = [
  'weekend', 
  'hour_type', 
  'trip_type', 
  'desc_user_type', 
  'desc_ageRange'
]


x_vars = ['travel_time', 'avg_speed']
alpha = 0.05
combi_vars = combinatoria(vars) 
for combi_var in combi_vars:
  if combi_var:
    dfTripsIQR['code'] = getCode(dfTripsIQR, combi_var)
    for x_var in x_vars:
      title = 'BoxPlot ' + '+'.join(combi_var) + '<->' + x_var
      generateBoxPlot(x_var, 'code', dfTripsIQR, title)
      '''
      title = 'T-Test ' + '+'.join(combi_var) + '<->' + x_var
      z_pvalue = []
      values = sorted(dfTripsIQR['code'].unique().tolist())
      #Añadimos un caracter para que no lo transforme a número
      x_values = ['·' + str(x) for x in values] 
      for i in range(len(values)):
        x_pvalue = []
        data1=dfTripsIQR.loc[dfTripsIQR['code']==values[i]][x_var]
        for j in range(len(values)):
          data2=dfTripsIQR.loc[dfTripsIQR['code']==values[j]][x_var]
          t_stat, p = stats.ks_2samp(data1, data2)
          x_pvalue.append(p)
        z_pvalue.append(x_pvalue)
      generateHeatMap(x_values, x_values, z_pvalue, title,
        [[0, '#2E29CD'], 
        [alpha, '#E91160'],
        [1, '#EB1010']
        ],
        dict(
          titleside = 'top',
          tickmode = 'array',
          tickvals = [0, alpha],
          ticktext = ['Diferents','Equals'],
          ticks = 'outside'
        )
      )
      '''

Output hidden; open in https://colab.research.google.com to view.

Tras analizar detenidamente las distribuciones de los boxplots con las diferentes conbinaciones de variables, se han visto ciertas dependencias o similitudes que pueden llegar a dar una primera visión de las posibles agrupaciones que se van a poder encontrar:

- Se ve una similitud entre los usuarios de edad desconocida y los que están entre 41-65 con bono Anual. Del mismo modo encontramos similitud entre los usuarios de bono Anual y los de edad comprendida entre 19-26 y 27-40.
- Se ve una similitud importante entre las horas de la mañana (7-12 y 12-18) para los usuarios de edad media (27-40 y 41-65). 
- Se ve una similitud importante entre el tipo de usuario Anual y las horas de la mañana (7-12 y 12-18).
- Se ve una similitud importante entre los usuarios ocasionales y de compañia y las horas de la mañana (7-12 y 12-18).
- Se ve una similitud importante entre los usuarios hacen viaje retiro-retiro entre las horas 7-12 y 12-18 y sobre el mismo horario los usuarios que salen del retiro.
- Se ve una similitud importante entre los usuarios que salen del retiro con edades comprendidas entre 19-26 y 27-40. Asi mismo también se ve similitud entre los usuarios que cogen y dejan la bici en la misma estación con edades comprendidas entre 27-40 y 41-65. Además los usuarios de edad desconocida salen del retiro tienen una similitud importante con los usuarios que salen del retiro con edades comprendidas entre 41-65.
- Se ve una similitud importante entre los usuarios con edades comprendidas entre 19-26 y 27-40 en dias laborables. Sobre los mismos días hay una similitud importante con los usuarios de edad desconocida y los de 41-65. Hay similitud entre los usuarios de edades entre 19-26 y 41-65 en el uso de bicimad el fin de semana.
- Se ve una similitud el uso del servicio en dias laborables entre las horas comprendidas de las 7-12 y 12-18.
- Se ve una similitud entre los usuarios viajan retiro-retiro tanto fin de semana como días laborables.














### Tabla contingencias

In [0]:
vars = ['day_of_week', 'weekend', 'hour_type', 'trip_type', 'desc_user_type', 'desc_ageRange']
z_pvalue = []
for varx in range(len(vars)):
  x_pvalue = []
  for vary in range (len(vars)):
    crosstab = pd.crosstab(dfTrips[vars[varx]], dfTrips[vars[vary]])
    cramerv = cramers_v(crosstab)
    if vary > varx:
      generateCrosstabBars(crosstab, vars[varx], vars[vary], True)
    x_pvalue.append(cramerv)
  z_pvalue.append(x_pvalue)

generateHeatMap(
    vars, 
    vars, 
    z_pvalue, 
    'Correlación Variables Categóricas',
    [
      [0, 'rgb(48,0,255)'], 
      [0.3, 'rgb(0, 255, 180)'],
      [0.6, 'rgb(186, 255, 0)'], 
      [1, 'rgb(255,45,0)']
    ],
    dict(
      titleside = 'top',
      tickmode = 'array',
      tickvals = [0, 0.3, 0.6, 1],
      ticktext = ['no dependiente','leve', 'intensa', 'dependiente'],
      ticks = 'outside'
    )
)    

- Se ve un uso mayoritario en las franjas de uso 7-12 y 12-18.
- El rango de edades que mayor uso hacen son 27-40 y 41-65. 
- Existe un aumento en el uso en las horas entre 7-12 para el rango de edad 41-65.
- Uso mayoritario de usuarios que salen del retiro con bono anual.
- Tienden a crecer los usuarios de edad desconocida el fin de semana.
- Crece el uso el fin de semana en el horario de 18-23 y baja el de 7-12.
- Se duplica el uso el fin de semana en los viajes de la misma estación con bono ocasional.
- Los usuarios de edad desconocida hacen mayor uso en viajes de misma estación.
- Crece el uso de edad desconocida durante las horas de 12-18 y 18-23.
- Hay más uso de usuarios viajan retiro-retiro o se van del retiro que los que hacen el viaje a la misma estación.
- Los usuarios con rango de edad 27-40 hacen un mayor uso en viajes retiro-retiro o saliendo del retiro, disminuyen a la mitad cuando son viajes misma estación.
- El uso de abono ocasional es casi por completo de usuarios con edad desconocida.
- Tras ir analizando todos los datos se ve un uso por usuarios de abono de compañia que denota que añaden a su abono a familiares como se puede ver en el gráfico de las variables por tipo de usuario y rango de edad.

Observando la matriz que muestra la relacion de dependencia que hay entre las variables categoricas, puede observarse por un lado una gran dependencia de las variables day_of_week y weekend. Además se encuentra una dependencia o correlación leve entre las varibles de tryp_type y desc_user_type. Existe una tercera depencia ya más moderada entre las variables de desc_ageRange y desc_user_type.

Por lo tanto las variables categoricas que se van a utilizar para describir el dataset en el modelado son:
- weekend
- hour_type
- trip_type
- desc_user_type
- desc_ageRange





#Clustering


##KMeans
El algoritmo KMeans en Python que viene de scikit-learn tiene el principal hándicap que no acepta variables categóricas. Hay mucha literatura de cómo transformar (one hot encoding, binary, label encoding, etc..) categóricas en numéricas pero cualquiera de ellas puede generar problemas al proveer de ordinalidad a dichas variables. Problema que se acrecenta en KMeans ya que utiliza la distancia euclídea para encontrar los centroides.

En cualquier caso, se realizará un estudio utilizando éste algoritmo. Lo primero que hay que hacer es escalar las numéricas y convertir a númericas las variables categóricas (usaremos la función **get_dummies** (one hot encoding). Utilizar ésta técnica hace que se creen tantas variables como valores tenga cada variable categórica. En consecuencia, parece acertado ejecutar un PCA.

###KMeans One Hot Encoding

In [0]:
#Creacion de Variables: scalar de las numericas y one_hot de las categoricas
from sklearn.preprocessing import MinMaxScaler
vars = ['weekend', 'hour_type', 'trip_type', 'desc_user_type', 'desc_ageRange']
df = pd.get_dummies(dfTripsIQR[vars])
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(
    dfTripsIQR[['travel_time', 'avg_speed']]), 
    columns=['travel_time', 'avg_speed']
)

X = pd.concat([df,df_scaled], axis=1)

In [0]:
#PCA
from sklearn.decomposition import PCA
#Fitting the PCA algorithm with our Data
pca = PCA().fit(X)

generateLineChar(
    [None], 
    [np.cumsum(pca.explained_variance_ratio_)], 
    ['X'],
    'PCA Varianza', 
    'Número de componentes', 
    'Varianza (%)'
)



Parece que el PCA recomienda un total de 7 componentes.

Nos faltaría encontrar el número de clústers apropiado. También, hay mucha literatura al respecto y no hay **fórmula** adecuada ya que el número de clústers obedece a otras razones como negocio, campaña a realizar, etc..

Una de las técnicas más usadas es **Elbow Curve** en donde se puede apreciar qué número parece recomendable, basado bien en las distancias a los centroides o el scoring calculado.

In [0]:
from sklearn.cluster import KMeans
Nc = range(2, 20)
models = [KMeans(n_clusters=i) for i in Nc]
score = [models[i].fit(X).score(X) for i in range(len(models))]

pca = PCA(n_components=11).fit(X)
Xpca = pca.transform(X)

score_pca = [models[i].fit(Xpca).score(Xpca) for i in range(len(models))]
generateLineChar(
    [[i for i in Nc], [i for i in Nc]], 
    [score, score_pca],
    ['X', 'X_PCA'],
    'Curva Elbow K-Means', 
    'Número de Clusters', 
    'Curva Elbow'
)


Sin embargo, usando todas las variables o incluso las variables PCA, la curva no indica un codo claro para discernir el número de clústers.

In [0]:
from sklearn.cluster import KMeans
Nc = range(2, 20)
models = [KMeans(n_clusters=i) for i in Nc]
score = [models[i].fit(df_scaled).score(df_scaled) for i in range(len(models))]
generateLineChar(
    [[i for i in Nc]], 
    [score],
    ['numerics'],
    'Curva Elbow K-Means Variables Numéricas', 
    'Número de Clusters', 
    'Curva Elbow'
)


Usando sólo variables numéricas, parece que el número de clústers es 5. La curva Elbow con variables numéricas es más **bonita** por lo que parece que se empieza a intuir que, quizás, podría realizarse el clustering sólo con las numéricas.

In [0]:
#KMeans con n=4
n=4
from sklearn.preprocessing import MinMaxScaler
vars = ['weekend', 'hour_type', 'trip_type', 'desc_user_type', 'desc_ageRange']
df = pd.get_dummies(dfTripsIQR[vars])
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(
    dfTripsIQR[['travel_time', 'avg_speed']]), 
    columns=['travel_time', 'avg_speed']
)

X = pd.concat([df,df_scaled], axis=1)
kMeans = KMeans(n_clusters=n).fit(X)
dfTripsIQR['cluster'] = kMeans.labels_

In [0]:
vars = ['weekend', 'hour_type', 'desc_user_type', 'desc_ageRange', 'trip_type']
dfTripsCluster = dfTripsIQR[vars + ['cluster']]
x, y, z = generateClusterMap(
    dfTripsCluster, 
    vars, 
    sorted(dfTripsCluster['cluster'].unique().tolist()), 
    'cluster'
  )
title = "Mapa Relación Cluster K-Means One Hot Encoding"
  
generateHeatMap(x, y, z, title, None, None) 

####Conclusión
Tras analizar el conjunto de resultados con lo visto anteriormente en el analisis explotario, se ve reflejado gran parte de las conclusiones. Se denota gran peso de aquellas variables con un porcentaje muy alto en el dataset. 

Según se ha podido ver en 3 de los clusters parten de una premisa de usuarios con bono anual que salen desde el retiro en días laborables. Les diferencia el rango de edad (41-65, unknown, 27-40) y los rangos horarios donde dos hacen más actividad entre 7-12 y 12-18 y el otro12-18 y 18-23. El cuarto cluster se ve más diferenciado ya que aunque prevalece el tipo de bono y que sale desde el retiro, lo hace en dias laborables y son mayoritariamente usuarios de edad desconocida.

###KMeans con variables numéricas

In [0]:
#KMeans n=4 Variables Numéricas
n=4
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(
    dfTripsIQR[['travel_time', 'avg_speed']]), 
    columns=['travel_time', 'avg_speed']
)
X = df_scaled
kMeans = KMeans(n_clusters=n).fit(X)
dfTripsIQR['cluster'] = kMeans.labels_

Aúnque en gran medida los resultados anteriores guardan bastante similitud con lo visto en el analisis exploratorio, lo conveniente sería probar solo con variables númericas como se comporta. Debido a que este algoritmo aplica los calculos teniendo en cuenta la distancia euclidea y por la forma de los datos no vendría a ser la más adecuada.

In [0]:
vars = ['weekend', 'hour_type', 'desc_user_type', 'desc_ageRange', 'trip_type']
dfTripsCluster = dfTripsIQR[vars + ['cluster']]
x, y, z = generateClusterMap(
    dfTripsCluster, 
    vars, 
    sorted(dfTripsCluster['cluster'].unique().tolist()), 
    'cluster'
  )
title = "Mapa Relación Cluster K-Means Numéricas"
  
generateHeatMap(x, y, z, title, None, None) 

####Conclusión
Tal y como se ha podido comprobar gran parte de las variables más descriptoras se han mantenido. Aunque en este caso se parte de una base en la que los cuatro clusters serian viajes de usuarios con bono anual durante la semana que salen del retiro. En este caso también lo que varia son algo las edades aunque hace mucho foco en los de edad desconocida y los rangos horarios. Tiene similitudes con las agrupaciones que se veian en el analisis exploratorio pero no hay variedad de agrupaciones, ya que se ven todos muy cerca. 

De modo que este metodo no sería del todo definitivo a la hora de poder describir el dataset tal y como se ha entendido a lo largo de este estudio. Para seguir concretando se probara con el modelo Gaussian Mixture

##Gaussian Mixture
Este algorito tiene la misma problematica que el KMeans con las variables categóricas, ademas de necesitar que tengan una distribución gausiana.

Aún asi, se va a disponer a probar los mismos casos que con el modelo anterior para poder hacer comparaciones en las coclusiones de cada uno de los casos.

###Gaussian Mixture con One Hot Encoding

In [0]:
from sklearn.preprocessing import MinMaxScaler
vars = ['weekend', 'hour_type', 'trip_type', 'desc_user_type', 'desc_ageRange']
df = pd.get_dummies(dfTripsIQR[vars])
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(
   dfTripsIQR[['travel_time', 'avg_speed']]), 
   columns=['travel_time', 'avg_speed']
)

X = pd.concat([df,df_scaled], axis=1)
pca = PCA(n_components=11).fit(X)
Xpca = pca.transform(X)

In [0]:
#Para encontrar el "codo" en GaussianMixture, se usan los valores Akaike
#https://es.wikipedia.org/wiki/Criterio_de_informaci%C3%B3n_de_Akaike
from sklearn.mixture import GaussianMixture
Nc = range(2, 20)
models = [GaussianMixture(
    n_components=i, 
    covariance_type='full', 
    random_state=45) for i in Nc]
models_fit = [models[i].fit(X) for i in range(len(models))]
score_bic = [models_fit[i].bic(X) for i in range(len(models))]
score_aic = [models_fit[i].aic(X) for i in range(len(models))]

pca = PCA(n_components=11).fit(X)
Xpca = pca.transform(X)

models_fit = [models[i].fit(Xpca) for i in range(len(models))]
score_pca_bic = [models_fit[i].bic(Xpca) for i in range(len(models))]
score_pca_aic = [models_fit[i].aic(Xpca) for i in range(len(models))]

generateLineChar(
    [[i for i in Nc], [i for i in Nc], [i for i in Nc], [i for i in Nc]], 
    [score_bic, score_pca_bic, score_aic, score_pca_aic],
    ['X BIC', 'X_PCA BIC', 'X AIC', 'X_PCA AIC'],
    'Curva Elbow GaussianMixture', 
    'Número de Clusters', 
    'Curva Elbow'
)


Para este caso también se va a tratar de ver si la curva de Elbow es indicador de que número de clusters ha de tener. 

Para poder pintar dicha curva se utiliza la métrica de AIC, maneja un solución de compromiso entre la bondad de ajuste del modelo y la complejidad del modelo. Este se basa en ofrecer una estimación relativa de la información perdida cuando se utiliza un modelo determinado para representar el proceso que genera los datos.

Tal como puede observarse en el gráfico de arriba, este tampoco es determinante para poder tomar una decisión del número de clusters basado en dicha métrica. Por lo tanto se procede a hacer un clustering al igual que en el modelo anterior, con 4 (que son los que se han ido deduciendo durante el analisis).

In [0]:
#GaussianMixture con n=4
n=4
from sklearn.preprocessing import MinMaxScaler
vars = ['weekend', 'hour_type', 'trip_type', 'desc_user_type', 'desc_ageRange']
df = pd.get_dummies(dfTripsIQR[vars])
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(
    dfTripsIQR[['travel_time', 'avg_speed']]), 
    columns=['travel_time', 'avg_speed']
)

X = pd.concat([df,df_scaled], axis=1)
gaussianMixture = GaussianMixture(
    n_components=n, 
    covariance_type='full', 
    random_state=45).fit(X)
dfTripsIQR['cluster'] = gaussianMixture.predict(X)

In [0]:
vars = ['weekend', 'hour_type', 'desc_user_type', 'desc_ageRange', 'trip_type']
dfTripsCluster = dfTripsIQR[vars + ['cluster']]
x, y, z = generateClusterMap(
    dfTripsCluster, 
    vars, 
    sorted(dfTripsCluster['cluster'].unique().tolist()), 
    'cluster'
  )
title = "Mapa Relación Cluster GaussianMixture One Hot Encoding"
  
generateHeatMap(x, y, z, title, None, None) 

####Conclusión
Como se puede observar tiene muchas similitudes con el resultado del KMeans con One Hot Encoding. Se ven cuatro clusters definidos por un mismo patros que luego varía según el rango de edad, muy marcado, y un rango horario. Se ven dos clusters de usuarios que con bono anual salen del retiro entre las 7-12 y 12-18 con rangos de edad que cuadrán totalmente tanto con anteriores modelos como lo visto en el analisis (27-40 y 41-65).

Lo mas diferencial en este caso son los usuarios que salen del retiro el fin de semana con bono anual y con un rango de edad desconocido.


###Gaussian Mixture con variables numericas

Se procede a probar el modelo bajo las mismás características que el anterior para poder comparar los resultados.

In [0]:
#GaussianMixture n=4 Variables Numéricas
n=4
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(
    dfTripsIQR[['travel_time', 'avg_speed']]), 
    columns=['travel_time', 'avg_speed']
)
X = df_scaled
gaussianMixture = GaussianMixture(
    n_components=n, 
    covariance_type='full', 
    random_state=45).fit(X)
dfTripsIQR['cluster'] = gaussianMixture.predict(X)

In [0]:
vars = ['weekend', 'hour_type', 'desc_user_type', 'desc_ageRange', 'trip_type']
dfTripsCluster = dfTripsIQR[vars + ['cluster']]
x, y, z = generateClusterMap(
    dfTripsCluster, 
    vars, 
    sorted(dfTripsCluster['cluster'].unique().tolist()), 
    'cluster'
  )
title = "Mapa Relación Cluster GaussianMixture Numéricas"
  
generateHeatMap(x, y, z, title, None, None) 

####Conclusión
Este resultado quizás sea el menos descriptor de todos los modelos que se han probado hasta el momento. Mantiene un patron muy marcado por el bono anual, salida desde el retiro y en dias laborables en un rando de edad mayoritariamente desconocido. 

Por lo que no se puede tomar este resultado como que describe en un alto porcentaje el dataset

##KModas
Tras ver dos modelos, los cuales muchas teorias indican que están más bien preparados para hacer cluster con variables numéricas. Y visto los resultados, se quiere contrastar con un modelo que sigue el mismo patron pero si esta preparado para entender variables categoricas.

Este modelo define los grupos en función del número de categorías coincidentes entre los puntos de datos. 


In [0]:
#KModes n=4. Sin necesidad de categorizar variables..
!pip install kmodes
import numpy as np
from kmodes.kmodes import KModes

n=4
vars = [
    'travel_time', 
    'avg_speed', 
    'weekend', 
    'hour_type', 
    'trip_type', 
    'desc_user_type', 
    'desc_ageRange'
]

X = dfTripsIQR[vars]
kMode = KModes(n_clusters=n, init='Huang').fit(X)
dfTripsIQR['cluster'] = kMode.labels_


In [0]:
vars = ['weekend', 'hour_type', 'desc_user_type', 'desc_ageRange', 'trip_type']
dfTripsCluster = dfTripsIQR[vars + ['cluster']]
x, y, z = generateClusterMap(
    dfTripsCluster, 
    vars, 
    sorted(dfTripsCluster['cluster'].unique().tolist()), 
    'cluster'
  )
title = "Mapa Relación Cluster KModes"
  
generateHeatMap(x, y, z, title, None, None) 

####Conclusión
El resultado del modelo confirma los resultados que se han obtenido en las pruebas anteriores. Denotan cuatro clusters bastante cercanos entre si con un patron definido por bono anual y salida desde el retiro. Donde en este caso si pierde algo de fuerza la variable de dia laboral aunque sigue siendo muy fuerte y dando como resultado 3 clusters en dia laborable y tan solo uno en fin de semana. En este caso toman un peso fuerte y decisivo los rangos horarios de 12-18 y 18-23. En cuanto al rango de edad la gran mayoria se situan en rango desconocido y tan solo uno señala 27-40 como edad descriptora con gran peso.

Por lo tanto es sencillo decir que sigue la misma linea descriptora que los anteriores modelos aún y este estar preparado para variables categoricas y en los otros ser más discutible.

##SNA

Se parte de la idea de comparar las distribuciones a través del **Code Group**, de tal manera que cada valor del **Code** se considera como un nodo de una red. A partir del **Kolmogorov-Smirnov**, si dos Codes tienen la misma distribución, podemos suponer que están unidos con un determinado peso proporcionado por el p_value. Una vez creada la red, a partir de los algoritmos de comunidad, se obtendrán todos los nodos que pertenecen a la misma comunidad, en definitiva, al mismo clúster.

Se generará, por tanto, una red en donde dos nodos estará unidos sí su distribución es igual (p_value>=alfa, siendo alfa 0.05)

Dado que existen dos variables númericas y, en consecuencia, dos distribuciones para cada code, disponemos de varios p_values para cada code, en consecuencia, será necesario **inventar** el peso asociado a los dos nodos de la red.

Una primera aproximación sería utilizando la media de los valores. Otra aproximación podría ser agregando las variables númericas en una sola y calcular los p_value sobre ella, por ejemplo, la variable **travel_space_avg = travel_time * avg_speed**.


In [0]:
#Generacion de un dataframe con los p_values
combi_var = [
  'weekend', 
  'hour_type', 
  'trip_type', 
  'desc_user_type', 
  'desc_ageRange'
]


dfTripsIQR['code'] = getCode(dfTripsIQR, combi_var)
x_vars = ['travel_time', 'avg_speed', 'travel_space_avg']
#dfNodes = generateRelationPValue(combi_var, x_vars, dfTripsIQR)
dfNodes = pd.read_csv(file + '.nodes', sep=';')
dfNodes.head()

,nodeA,nodeB,p_travel_time,p_avg_speed,p_travel_space_avg
0,L07FA00,L07FA17,0.017428,0.124856,0.049817
1,L07FA00,L07FA19,0.087699,0.281000,0.211719
2,L07FA00,L07FA27,0.106987,0.153360,0.259418
3,L07FA00,L07FA41,0.109744,0.083076,0.289850
4,L07FA00,L07FA66,0.021274,0.014899,0.057137


In [0]:
#Inicializacion de variables Networks
Gs = {
    'mean': {'G': nx.Graph(), 'partition': None},
    'space': {'G': nx.Graph(), 'partition': None}
}

### Media travel_time y avg_speed

In [0]:
#Creacion de la red utilizando como peso la media de ambos p_values
alpha = 0.05
for index, row in dfNodes.iterrows():
  p = (row['p_travel_time'] + row['p_avg_speed'])/2
  if p> alpha:
    Gs['mean']['G'].add_edge(row['nodeA'], row['nodeB'], weight=p)
nx.write_gml(Gs['mean']['G'], file + '.mean.gml')

### Uso de variable travel_space_avg

In [0]:
#Creacion de la red utilizando como peso la media de ambos p_values
alpha = 0.05
for index, row in dfNodes.iterrows():
  p = row['p_travel_space_avg']
  if p> alpha:
    Gs['space']['G'].add_edge(row['nodeA'], row['nodeB'], weight=p)
nx.write_gml(Gs['space']['G'], file + '.space.gml')

### Información de la Red

In [0]:
for g in Gs:
  #G=nx.read_gml(, file + '.' + g + '.gml')
  G = Gs[g]['G']
  print("Network " + g + "----------------------------------------")
  print('nodes: ',G.order(),'; size: ',G.size())


  print("eccentricity: %s" % nx.eccentricity(G))
  print("radius: %d" % nx.radius(G))
  print("diameter: %d" % nx.diameter(G))
  print("center: %s" % nx.center(G)) #Nodos que estan en el centro (que estan a distancia radius)
  print("periphery: %s" % nx.periphery(G)) #Nodos mas alejados (que estan a distancia diameter)
  print("density: %s" % nx.density(G))
  print("---------------------------------------------------------")


Network mean----------------------------------------
nodes:  220 ; size:  18537
eccentricity: {'L07FA00': 2, 'L07FA17': 2, 'L07FA19': 2, 'L07FA27': 2, 'L07FA41': 2, 'L07FAUn': 2, 'L07FC00': 2, 'L07FC19': 2, 'L07FC27': 2, 'L07FC41': 2, 'L07FCUn': 2, 'L07RA19': 2, 'L07RA27': 2, 'L07RA41': 2, 'L07RA66': 2, 'L07RAUn': 2, 'L07RC41': 1, 'L07RCUn': 2, 'L07SA19': 2, 'L07SA41': 2, 'L07SA66': 2, 'L07SC00': 1, 'L07SC19': 2, 'L07SC41': 2, 'L07SOUn': 2, 'L12FA00': 2, 'L12FA17': 2, 'L12FA19': 2, 'L12FA27': 2, 'L12FA41': 2, 'L12FAUn': 2, 'L12FC00': 2, 'L12FC27': 2, 'L12FC41': 2, 'L12FCUn': 2, 'L12RA00': 2, 'L12RA17': 1, 'L12RA19': 2, 'L12RA27': 2, 'L12RA41': 2, 'L12RA66': 2, 'L12RAUn': 2, 'L12ROUn': 2, 'L12SA19': 1, 'L12SA66': 1, 'L12SC27': 1, 'L18FA00': 2, 'L18FA17': 2, 'L18FA19': 2, 'L18FA27': 2, 'L18FA41': 2, 'L18FA66': 2, 'L18FAUn': 2, 'L18FC00': 2, 'L18FC19': 1, 'L18FC27': 2, 'L18FCUn': 2, 'L18RA00': 1, 'L18RA17': 2, 'L18RA19': 2, 'L18RA27': 2, 'L18RA41': 2, 'L18RAUn': 2, 'L18RC27': 1, 'L18SA19'

### Generación de comunidades

In [0]:
import community as co #Usamos esta libreria para las comunidades
for g in Gs:
  G = Gs[g]['G']
  Gs[g]['partition']=co.community_louvain.best_partition(G) 
  print("Network " + g + '->', 'nodes: ', G.order(),
    '; communities: ',len(set(Gs[g]['partition'].values())))


Network mean-> nodes:  220 ; communities:  4
Network space-> nodes:  220 ; communities:  3


In [0]:
vars = ['weekend', 'hour_type', 'desc_user_type', 'desc_ageRange', 'trip_type']
for g in Gs:
  partition = Gs[g]['partition']
  dfComs=pd.DataFrame({
      'node':list(partition.keys()),
      'com':list(partition.values())
  })
  dfTripsCluster = dfTripsIQR.merge(
      dfComs, 
      left_on='code', 
      right_on='node', 
      how='inner'
  )
  x, y, z = generateClusterMap(
    dfTripsCluster, 
    vars, 
    dfTripsCluster['com'].unique().tolist(), 
    'com'
  )
  title = "Mapa Relación Cluster SNA"
  if g=='mean':
    title+=": Media travel_time y avg_speed"
  else:
    title+=": travel_space_avg"
  generateHeatMap(x, y, z, title, None, None) 



###Conclusión
Tal y como puede verse en los resultados, estos son donde los clusters estan más definidos y con una mayor distancia entre ellos. De cara a una posible predicción estaría mucho más claro a cual debería ir el nuevo viaje. 

En el primero de los casos, donde se calcula la media de los valores de ambas variables, indica tal y como se viene observando hasta ahora cuatro clusters. En este caso a diferencia del resto a sido totalmente inducido por el modelo. Donde diferencia fuertemente por usuarios con bono anual o de compañia, y si sale desde el retiro o no. 

Por lo tanto estos clusters serian:
- Viajes en días laborables con bono anual con edades comprendidas entre 27-40 y desconocidas. En unos horarios de 12-18 sobre todo.
- Viajes en días laborables con una mayoria de bono de compañia con una edad comprendida entre 27-40 en horario de 7-12 y 12-18.
- Viajes en días laborables mayoritariamente con bono anual y edad desconocida en un horario mucho más disperso y equilibrado.
- Usuarios con bono de compañia que salen y dejan la bici en la misma estación en un horario de 7-12 con edades comprendidas entre 27-40 y 41-65.

Teniendo en cuenta la otra alternativa con travel_space_avg, el resultado se reduce a 3 clusters. Estos están más cerca que los anteriores y se intuye que no describen el mismo porcentaje que el anterior, si no menos. Dos de los clusters son bien similares al anterior resultado y el tercero en una variación donde son usuarios que viajan desde el retiro en día laboral con abono ocasional y edad desconocida en un horario muy marcado de 12-18.



##Conclusión

Tras analizar el dataset, y ver como se han comportado todos los modelos se llegan a comprender varias cosas.
- El analisis inicial se encuentra representado en gran media por gran parte de los modelos.
- Las variables categoricas incluso en modelos discutidos por su inclusión han aportado información que ha ayudado a describir mejor el dataset.
- Con todo ello es cierto que hay parte quizás por el bajo volumen de samples que no llegan a estar quizás debidamente representados.
- El modelo de SNA es quién describe mejor mayor parte del dataset sin dejar de representar lo intuido en el analisis.

En cuanto al resultado en total de todos los modelos se pueden inferir los siguientes dos grupos más definidos y otros dos algo no tan claros:
- Viajes de usuarios entre 27-40 y 41-65 que salen desde el retiro en un horario de 7-12 y 12-18 con bono anual.
- Viajes de usuarios entre 41-65y rango desconocido que salen en horario de 12-18 y 18-23 con bono en mayor medida anual.
- Viajes de usuarios de fin de semana con bono anual mayoritariamente y en un rango horario comprendido entre 12-18 y 18-23.
- Viajes de usuarios de compañia en días laborables y rangos mayoritariamente entre 27-40 y 41-65.

Estos dos últimos estarían menos claros su definición, lo que termina dificultando el tener un dibujo definitivo del dataset. Por lo que a este estudio le tendrían que seguir acciones futuras para tratar de comprender mejor los datos. 
- Habría que investigar que señal darían los datos si se quitase el rango de edad Unknown que es un alto porcentaje y directamente no esta aportando demasiada información. 
- Además habría que valorar que resultados salen restandole peso descriptivo con tecnicas de balanceo o incluso quitando del dataset a la variable desc_user_type que contiene más de un 90% de samples de bono anual.
- Se valoraría agregar más la variable de rango de edad ya que las franjas no estan niveladas y quizás con menos se obtendría mejor señal.
- Se valoraría estudiar más modelos que manejen variables categoricas para ver si dictan resultados más claros.
